## Exercícios de Pesquisa Operacional

Exercícios criados projetados para testar competências essenciais, tais como:
* Otimização linear;
* Tomada de decisão em cenários reais; e
* Estratégias práticas de logística e gestão.

### Prepara o ambiente de execução

In [2]:
!pip install pulp | tail -1
!pip install pandas | tail -1
!pip install numpy | tail -1

import pulp
import pandas as pd
import numpy as np

### Exercício 1: Problema de Otimização de Rotas

Uma empresa de transportes possui 5 centros de distribuição que devem realizar entregas diárias em 12 pontos da cidade. Considere que cada centro tem uma capacidade máxima diária para atender até 3 pontos.

---

* Monte um modelo matemático de otimização linear inteiro que minimize a distância total percorrida pelos veículos, garantindo o atendimento a todos os pontos com a capacidade estabelecida.

* Utilize os dados da tabela "data/distancias_sudeste.csv"

In [6]:
df = pd.read_csv('data/distancias_sudeste.csv')

# renomeia primeira coluna
df.rename(columns={'Unnamed: 0': 'CD'}, inplace=True)

df.head()

,CD,Santos,Niteroi,Juiz de Fora,Uberlandia,Sorocaba,Ribeirao Preto,Volta Redonda,Petropolis,Governador Valadares,Campos dos Goytacazes,Jundiai,Sao Jose dos Campos
0,Sao Paulo,80,430,500,580,90,320,310,450,900,710,60,90
1,Rio de Janeiro,500,15,180,850,530,720,130,70,580,280,480,340
2,Belo Horizonte,600,440,260,540,630,480,340,370,320,530,520,510
3,Vitória,900,520,470,910,950,980,470,440,330,230,870,810
4,Campinas,170,500,510,470,90,220,400,490,750,610,40,130


In [21]:
# separa as distancias em um dataFrame
distancias = df.iloc[:, 1:]
distancias.head()

# Cria o Modelo de Otimização
model = pulp.LpProblem("Rotas", pulp.LpMinimize)

# Cria as variáveis de decisão Xij, onde X define se o CD "i" irá passar pela cidade "j"
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(1, 6) for j in range(1, 13)), cat='Binary')

# Define a função objetivo -> soma das distancias percorridas
model += pulp.lpSum(distancias.iloc[i-1, j-1] * x[(i, j)] for i in range(1, 6) for j in range(1, 13))

# Define as restrições ao modelo

# Restrição 1: Deve passar por todas as cidades
for j in range(1, 13):
    model += pulp.lpSum(x[(i, j)] for i in range(1, 6)) == 1

# Restrição 2: Cada Centro de Distribuição tem capacidade de enviar até 3 cidades
for i in range(1, 6):
    model += pulp.lpSum(x[(i, j)] for j in range(1, 13)) <= 3

# Resolve o modelo
model.solve()

1

In [40]:
# Resultados
for i, cd in enumerate(df.iloc[:,0]):
    distancia_total = 0
    print(f"Centro de Distribuição {cd}:\t")
    for j, ponto in enumerate(df.columns[1:]):
        if x[(i+1, j+1)].varValue == 1:
            distancia_total += distancias.iloc[i, j]
            print(f"\t - {ponto}")
    print(f"\t Distância (km): {distancia_total}")

print("Distância Total (km): ", pulp.value(model.objective))

Centro de Distribuição Sao Paulo:	
	 - Santos
	 - Sorocaba
	 - Sao Jose dos Campos
	 Distância (km): 260
Centro de Distribuição Rio de Janeiro:	
	 - Niteroi
	 - Volta Redonda
	 - Petropolis
	 Distância (km): 215
Centro de Distribuição Belo Horizonte:	
	 - Juiz de Fora
	 - Governador Valadares
	 Distância (km): 580
Centro de Distribuição Vitória:	
	 - Campos dos Goytacazes
	 Distância (km): 230
Centro de Distribuição Campinas:	
	 - Uberlandia
	 - Ribeirao Preto
	 - Jundiai
	 Distância (km): 730
Distância Total (km):  2015.0
